In [58]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

calendar = pd.read_csv('calendar_seattle.csv')
listings = pd.read_csv('listings_seattle.csv')
reviews = pd.read_csv('reviews_seattle.csv')
#reviews.head()
#calendar.head()
#listings.head()

#list_cal=pd.merge(listings, calendar, left_on="id", right_on="listing_id")
num_vars = listings.select_dtypes(include=['float', 'int']).columns
#num_vars
cat_vars = listings.select_dtypes(include=['object']).columns
cat_vars


Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'calendar_updated', 'has_availability',
       'calendar_last_s

In [ ]:
listings[['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'calendar_updated', 'has_availability',
       'calendar_last_scraped', 'first_review', 'last_review',
       'requires_license', 'jurisdiction_names', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification']].head()

In [ ]:
listings[['id', 'scrape_id', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'reviews_per_month']].head()


In [50]:
#for col in num_vars:
#        df[col].fillna((df[col].mean()), inplace=True)
most_missing_cols = set(listings.columns[listings.isnull().mean() > 0.75])
most_missing_cols



{'license', 'square_feet'}

In [23]:
listings.columns[listings.isna().all()==True]

Index(['license'], dtype='object')

In [43]:
#listings.columns[listings.columns.value_counts()==1].tolist() 
for col in listings.columns:
  value_c = listings[col].nunique() ==1
  if value_c == True:
      print(col)
#listings.columns[listings[col].nunique() ==1]

scrape_id
last_scraped
experiences_offered
market
country_code
country
has_availability
calendar_last_scraped
requires_license
jurisdiction_names


In [59]:
 #Drop na and contains only one value in columns
listings = listings.drop(['scrape_id','last_scraped','experiences_offered','market','country_code',
'country','has_availability','calendar_last_scraped','requires_license','jurisdiction_names','license'], axis=1)

In [62]:
listings.price

0        $85.00
1       $150.00
2       $975.00
3       $100.00
4       $450.00
5       $120.00
6        $80.00
7        $60.00
8        $90.00
9       $150.00
10       $95.00
11       $99.00
12      $245.00
13      $165.00
14      $461.00
15      $109.00
16       $66.00
17       $99.00
18      $200.00
19      $700.00
20      $110.00
21       $99.00
22      $450.00
23       $75.00
24      $600.00
25      $450.00
26      $150.00
27       $45.00
28      $300.00
29       $80.00
         ...   
3788    $115.00
3789     $60.00
3790    $175.00
3791     $85.00
3792    $109.00
3793    $109.00
3794    $107.00
3795    $200.00
3796     $85.00
3797     $70.00
3798     $77.00
3799     $65.00
3800    $185.00
3801    $108.00
3802    $200.00
3803    $250.00
3804    $125.00
3805    $100.00
3806    $160.00
3807    $109.00
3808    $175.00
3809    $120.00
3810    $154.00
3811     $65.00
3812     $95.00
3813    $359.00
3814     $79.00
3815     $93.00
3816     $99.00
3817     $87.00
Name: price, Length: 381

In [54]:
listings.head()

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,thumbnail_url,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,NaN,NaN,NaN,NaN,...,10.0,10.0,9.0,10.0,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,...,10.0,10.0,10.0,10.0,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,NaN,...,10.0,10.0,10.0,10.0,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,We are in the beautiful neighborhood of Queen ...,Belltown,The nearest public transit bus (D Line) is 2 b...,NaN,...,10.0,10.0,9.0,9.0,f,strict,f,f,1,0.89


In [ ]:
""" remove all the below:
    listing_url - url 
    name - textual description 
    summary - same as  above
    space - as above
    description - as above
    notes - as above
    transit - as above
    thumbnail_url - url
    medium_url - url
    picture_url - url
    xl_picture_url - url
    host_id - id that is not used anywhere else
"""


In [61]:
listings[[ 'listing_url' ,'name',  'summary' ,
    'space',
    'description', 
    'notes', 
    'transit' ,
    'thumbnail_url' ,
    'medium_url', 
    'picture_url', 
   'xl_picture_url', 
    'host_id']]

,listing_url,name,summary,space,description,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id
0,https://www.airbnb.com/rooms/241032,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883
1,https://www.airbnb.com/rooms/953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328
2,https://www.airbnb.com/rooms/3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,NaN,NaN,https://a2.muscache.com/ac/pictures/b4324e0f-a...,NaN,16708587
3,https://www.airbnb.com/rooms/7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,NaN,NaN,NaN,NaN,https://a0.muscache.com/ac/pictures/94146944/6...,NaN,9851441
4,https://www.airbnb.com/rooms/278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Belltown,The nearest public transit bus (D Line) is 2 b...,NaN,NaN,https://a1.muscache.com/ac/pictures/6120468/b0...,NaN,1452570
5,https://www.airbnb.com/rooms/5956968,Private unit in a 1920s mansion,We're renting out a small private unit of one ...,If you include a bit of your background in you...,We're renting out a small private unit of one ...,Let me know if you need anything or have sugge...,The #1 and #2 bus picks up 2-3 blocks away (di...,https://a2.muscache.com/ac/pictures/84791950/f...,https://a2.muscache.com/im/pictures/84791950/f...,https://a2.muscache.com/ac/pictures/84791950/f...,https://a2.muscache.com/ac/pictures/84791950/f...,326758
6,https://www.airbnb.com/rooms/1909058,Queen Anne Private Bed and Bath,Enjoy a quiet stay in our comfortable 1915 Cra...,Enjoy a quiet stay in our comfortable 1915 Cra...,Enjoy a quiet stay in our comfortable 1915 Cra...,The room now has a mini frig to keep your favo...,The bus goes through the lower Queen Anne neig...,https://a2.muscache.com/ac/pictures/26505642/2...,https://a2.muscache.com/im/pictures/26505642/2...,https://a2.muscache.com/ac/pictures/26505642/2...,https://a2.muscache.com/ac/pictures/26505642/2...,2497928
7,https://www.airbnb.com/rooms/856550,Tiny Garden cabin on Queen Anne,"Our tiny cabin is private , very quiet and com...","This cabin was built with Airbnb in mind, Que...","Our tiny cabin is private , very quiet and com...",There are three rentals in our back yard . If ...,"the ""D"" line runs out in front of our house .",https://a2.muscache.com/ac/pictures/83538305/d...,https://a2.muscache.com/im/pictures/83538305/d...,https://a2.muscache.com/ac/pictures/83538305/d...,https://a2.muscache.com/ac/pictures/83538305/d...,4016632
8,https://www.airbnb.com/rooms/4948745,Urban Charm || Downtown || Views,"Nestled in the heart of the city, this space i...","Located in the heart of the city, this space i...","Nestled in the heart of the city, this space i...",NaN,Bus stop to downtown directly across the stree...,https://a0.muscache.com/ac/pictures/85453284/d...,https://a0.muscache.com/im/pictures/85453284/d...,https://a0.muscache.com/ac/pictures/85453284/d...,https://a0.muscache.com/ac/pictures/85453284/d...,2166277
9,https://www.airbnb.com/rooms/2493658